<a href="https://colab.research.google.com/github/YianKim/2022_PythonStudy_face_emotion_recognition/blob/main/uncertainty_%EA%B8%B0%EB%B0%98_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글 드라이브


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 불러오기


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import PIL
import pickle
import random
from tqdm import tqdm
from collections import Counter

from keras.layers.core import Lambda
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import LSTM
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import *
from keras.models import load_model

from tqdm import tqdm

## 데이터 불러오기


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(data, labels), (test_data, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_data = data[range(10000)].reshape([10000,28,28,1])
train_labels = labels[range(10000)].reshape([10000,1])

In [ ]:
unlab_data = data[range(10000,60000)].reshape([50000,28,28,1])
unlab_labels = labels[range(10000,60000)].reshape([50000,1])

In [ ]:
test_data = test_data.reshape([10000,28,28,1])
test_labels = test_labels.reshape([10000,1])

In [ ]:
train_labels2 = []
unlab_labels2 = []
test_labels2 = []

In [ ]:
for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[train_labels[i][0]] = 1
  train_labels2.append(white)

for i in range(50000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[unlab_labels[i][0]] = 1
  unlab_labels2.append(white)

for i in range(10000):
  white = [0,0,0,0,0,0,0,0,0,0]
  white[test_labels[i][0]] = 1
  test_labels2.append(white)

In [ ]:
train_labels = np.array(train_labels2)
unlab_labels = np.array(unlab_labels2)
test_labels = np.array(test_labels2)

## CNN

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
110/110 [==============================] - 15s 18ms/step - loss: 3.2566 - accuracy: 0.5900 - val_loss: 0.8345 - val_accuracy: 0.7040 - lr: 0.0010
Epoch 2/2000
110/110 [==============================] - 2s 14ms/step - loss: 0.6978 - accuracy: 0.7519 - val_loss: 0.6845 - val_accuracy: 0.7440 - lr: 0.0010
Epoch 3/2000
110/110 [==============================] - 1s 13ms/step - loss: 0.5848 - accuracy: 0.7866 - val_loss: 0.6348 - val_accuracy: 0.7847 - lr: 0.0010
Epoch 4/2000
110/110 [==============================] - 2s 14ms/step - loss: 0.5179 - accuracy: 0.8086 - val_loss: 0.6325 - val_accuracy: 0.7720 - lr: 0.0010
Epoch 5/2000
110/110 [==============================] - 2s 14ms/step - loss: 0.4800 - accuracy: 0.8259 - val_loss: 0.6128 - val_accuracy: 0.7900 - lr: 0.0010
Epoch 6/2000
110/110 [==============================] - 1s 13ms/step - loss: 0.4445 - accuracy: 0.8356 - val_loss: 0.5553 - val_accuracy: 0.8027 - lr: 0.0010
Epoch 7/2000
110/110 [=============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

## 모델 결과

In [ ]:
def model_eval():
  pred_mu = model.predict(test_data)
  for i in range(1, 30):
    pred_mu += model.predict(test_data)
  pred_mu = pred_mu/30
  predicted_test_labels = np.argmax(pred_mu, axis=1)
  return(accuracy_score(np.argmax(test_labels, axis=1), predicted_test_labels))

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')
model_eval()

0.8852

# 라벨링

## 라벨링; 1

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn.h5')

In [ ]:
# Vars : 분산들
# Outs : 결과 값들

Vars = []
Outs = []

temp1 = []

for i in tqdm(range(30)):
  temp1.append(model.predict(unlab_data))

for j in tqdm(range(unlab_data.shape[0])):

  temp2 = np.array([0,0,0,0,0,0,0,0,0,0], 'float32')
  
  for i in range(30):
    temp2 += temp1[i][j]
  Outs.append(temp2/30)
  
  outputs=[]

  for i in range(30):
    outputs.append(temp1[i][j][np.argmax(temp2)])
  Vars.append(np.var(outputs))

100%|██████████| 50000/50000 [00:10<00:00, 4889.33it/s]


In [ ]:
Outs = np.array(Outs)

class마다 균등하게 선택

In [ ]:
Counter(np.argmax(unlab_labels, axis=1))

Counter({0: 5058,
         1: 4973,
         2: 4984,
         3: 4981,
         4: 5026,
         5: 5011,
         6: 4979,
         7: 4978,
         8: 5010,
         9: 5000})

In [ ]:
# upper bound of variance?

UB = []

for h in range(10):
  classvars = []
  for i in tqdm(range(50000)):
    if np.argmax(unlab_labels, axis=1).tolist()[i]==h:
      classvars.append(Vars[i])
  UB.append(np.percentile(classvars, 50))

100%|██████████| 50000/50000 [01:24<00:00, 589.32it/s]


In [ ]:
lowvars = []
ind = 0 

for i in Vars:
  if i <= UB[np.argmax(unlab_labels, axis=1)[ind]]:
    lowvars.append(ind)
  ind += 1

In [ ]:
highvars = []
for i in range(unlab_data.shape[0]):
  if i not in lowvars:
    highvars.append(i)

In [ ]:
# 같은 길이의 랜덤하게 고른 data들에 모델로 label 부여 후 정확도 측정
randomindx = random.sample(range(50000), 50000)
randomindx2 = randomindx[0:25000]
randomindx = randomindx[25000:50000]
accuracy_score(np.argmax(np.array(Outs)[randomindx], axis=1), np.argmax(unlab_labels[randomindx], axis=1))

0.89192

In [ ]:
# 저분산의 data들만 모아서 모델로 label 부여 후 정확도 측정 : 95% 이상
accuracy_score(np.argmax(np.array(Outs)[lowvars], axis=1), np.argmax(unlab_labels[lowvars], axis=1))

0.9728878433874247

In [ ]:
train_data_1 = np.concatenate([train_data, unlab_data[lowvars]], axis=0)
train_labels_1 = np.concatenate([train_labels, Outs[lowvars]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_1.shape[0]),train_data_1.shape[0])
train_data_1 = train_data_1[shufindx]
train_labels_1 = train_labels_1[shufindx]

In [ ]:
train_data_2 = np.concatenate([train_data, unlab_data[randomindx]], axis=0)
train_labels_2 = np.concatenate([train_labels, Outs[randomindx]], axis=0)

In [ ]:
shufindx = random.sample(range(train_data_2.shape[0]),train_data_2.shape[0])
train_data_2 = train_data_2[shufindx]
train_labels_2 = train_labels_2[shufindx]

In [ ]:
unlab_data_1 = unlab_data[highvars]
unlab_labels_1 = unlab_labels[highvars]
unlab_data_2 = unlab_data[randomindx2]
unlab_labels_2 = unlab_labels[randomindx2]

In [ ]:
Counter(np.argmax(train_labels_1, axis=1))

Counter({0: 3600,
         1: 3599,
         2: 3613,
         3: 3638,
         4: 3469,
         5: 3491,
         6: 3151,
         7: 3507,
         8: 3505,
         9: 3508})

In [ ]:
Counter(np.argmax(train_labels_2, axis=1))

Counter({0: 3409,
         1: 3473,
         2: 3582,
         3: 3546,
         4: 3512,
         5: 3488,
         6: 3448,
         7: 3581,
         8: 3447,
         9: 3514})

In [ ]:
Counter(np.argmax(unlab_labels_1, axis=1))

Counter({0: 2529,
         1: 2407,
         2: 2492,
         3: 2490,
         4: 2513,
         5: 2505,
         6: 2489,
         7: 2489,
         8: 2505,
         9: 2500})

In [ ]:
Counter(np.argmax(unlab_labels_2, axis=1))

Counter({0: 2565,
         1: 2496,
         2: 2452,
         3: 2497,
         4: 2506,
         5: 2458,
         6: 2504,
         7: 2474,
         8: 2554,
         9: 2494})

## 모델링

간단한 모델에서 받은 지식를 복잡한 모델에서 학습 

uncertainty aware data vs random sampling data

In [ ]:
def PermaDropout(rate):
    return Lambda(lambda x: K.dropout(x, level=rate))

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
384/384 [==============================] - 11s 24ms/step - loss: 1.0012 - accuracy: 0.6886 - val_loss: 0.5072 - val_accuracy: 0.8266 - lr: 0.0010
Epoch 2/2000
384/384 [==============================] - 9s 22ms/step - loss: 0.4127 - accuracy: 0.8674 - val_loss: 0.3816 - val_accuracy: 0.8791 - lr: 0.0010
Epoch 3/2000
384/384 [==============================] - 8s 22ms/step - loss: 0.3427 - accuracy: 0.8944 - val_loss: 0.3175 - val_accuracy: 0.9002 - lr: 0.0010
Epoch 4/2000
384/384 [==============================] - 8s 22ms/step - loss: 0.2910 - accuracy: 0.9115 - val_loss: 0.2988 - val_accuracy: 0.9161 - lr: 0.0010
Epoch 5/2000
384/384 [==============================] - 9s 22ms/step - loss: 0.2704 - accuracy: 0.9202 - val_loss: 0.2644 - val_accuracy: 0.9214 - lr: 0.0010
Epoch 6/2000
384/384 [==============================] - 8s 22ms/step - loss: 0.2438 - accuracy: 0.9299 - val_loss: 0.2580 - val_accuracy: 0.9280 - lr: 0.0010
Epoch 7/2000
384/384 [=============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model_eval()

0.8938

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
383/383 [==============================] - 10s 23ms/step - loss: 1.2074 - accuracy: 0.6101 - val_loss: 0.6716 - val_accuracy: 0.7693 - lr: 0.0010
Epoch 2/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.6132 - accuracy: 0.7924 - val_loss: 0.5614 - val_accuracy: 0.8164 - lr: 0.0010
Epoch 3/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.5290 - accuracy: 0.8287 - val_loss: 0.5035 - val_accuracy: 0.8420 - lr: 0.0010
Epoch 4/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4836 - accuracy: 0.8504 - val_loss: 0.4638 - val_accuracy: 0.8570 - lr: 0.0010
Epoch 5/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4430 - accuracy: 0.8628 - val_loss: 0.4506 - val_accuracy: 0.8702 - lr: 0.0010
Epoch 6/2000
383/383 [==============================] - 8s 22ms/step - loss: 0.4222 - accuracy: 0.8691 - val_loss: 0.4217 - val_accuracy: 0.8756 - lr: 0.0010
Epoch 7/2000
383/383 [=============================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')
model_eval()

0.8966

# 남은 데이터 라벨링하기


In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn2.h5')
model2 = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no.h5')

In [ ]:
unlab_label_1 = model1.predict(unlab_data_1)

In [ ]:
unlab_label_2 = model2.predict(unlab_data_2)

In [ ]:
train_data_1 = np.concatenate([train_data_1, unlab_data_1], axis=0)
train_labels_1 = np.concatenate([train_labels_1, unlab_label_1], axis=0)

In [ ]:
train_data_2 = np.concatenate([train_data_2, unlab_data[lowvars]], axis=0)
train_labels_2 = np.concatenate([train_labels_2, Outs[lowvars]], axis=0)

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_1,
          train_labels_1,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
657/657 [==============================] - 16s 23ms/step - loss: 0.9943 - accuracy: 0.6618 - val_loss: 0.9468 - val_accuracy: 0.6839 - lr: 0.0010
Epoch 2/2000
657/657 [==============================] - 18s 27ms/step - loss: 0.4825 - accuracy: 0.8375 - val_loss: 0.7981 - val_accuracy: 0.7357 - lr: 0.0010
Epoch 3/2000
657/657 [==============================] - 15s 22ms/step - loss: 0.4059 - accuracy: 0.8680 - val_loss: 0.7622 - val_accuracy: 0.7401 - lr: 0.0010
Epoch 4/2000
657/657 [==============================] - 15s 22ms/step - loss: 0.3596 - accuracy: 0.8887 - val_loss: 0.7200 - val_accuracy: 0.7722 - lr: 0.0010
Epoch 5/2000
657/657 [==============================] - 15s 22ms/step - loss: 0.3328 - accuracy: 0.9004 - val_loss: 0.7096 - val_accuracy: 0.7649 - lr: 0.0010
Epoch 6/2000
657/657 [==============================] - 15s 22ms/step - loss: 0.3105 - accuracy: 0.9095 - val_loss: 0.6970 - val_accuracy: 0.7839 - lr: 0.0010
Epoch 7/2000
657/657 [========================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn3.h5')
model_eval()

0.8951

In [ ]:
#######HYPERPARAMATERS###########
epochs = 2000
batch_size = 64
learning_rate = 0.001
#################################
  
model = Sequential()
    
model.add(Conv2D(32, kernel_size = (3,3), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Conv2D(128,(3,3), activation='relu'))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(PermaDropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(PermaDropout(0.5))

model.add(Dense(10, activation='softmax'))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
# print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=30, mode='auto')
checkpointer = ModelCheckpoint('weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)


model.fit(
          train_data_2,
          train_labels_2,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.3,
          shuffle = True,
          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/2000
658/658 [==============================] - 16s 23ms/step - loss: 1.0818 - accuracy: 0.6303 - val_loss: 0.3992 - val_accuracy: 0.8598 - lr: 0.0010
Epoch 2/2000
658/658 [==============================] - 15s 22ms/step - loss: 0.5546 - accuracy: 0.8151 - val_loss: 0.2712 - val_accuracy: 0.9131 - lr: 0.0010
Epoch 3/2000
658/658 [==============================] - 15s 23ms/step - loss: 0.4773 - accuracy: 0.8430 - val_loss: 0.2152 - val_accuracy: 0.9344 - lr: 0.0010
Epoch 4/2000
658/658 [==============================] - 15s 22ms/step - loss: 0.4363 - accuracy: 0.8662 - val_loss: 0.1783 - val_accuracy: 0.9547 - lr: 0.0010
Epoch 5/2000
658/658 [==============================] - 15s 23ms/step - loss: 0.4078 - accuracy: 0.8799 - val_loss: 0.1647 - val_accuracy: 0.9578 - lr: 0.0010
Epoch 6/2000
658/658 [==============================] - 15s 22ms/step - loss: 0.3868 - accuracy: 0.8877 - val_loss: 0.1529 - val_accuracy: 0.9650 - lr: 0.0010
Epoch 7/2000
658/658 [========================

In [ ]:
model.save('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/파이썬스터디 프로젝트/labeling_cnn_no2.h5')
model_eval()

0.8958